In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
import pyperclip
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import requests as req
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
  # 웹드라이버 실행
driver = webdriver.Chrome()

# 제품 목록 페이지 로드
driver.get("https://www.flipkart.com/televisions/pr?sid=ckf%2Cczl&p%5B%5D=facets.availability%255B%255D%3DExclude%2BOut%2Bof%2BStock&otracker=categorytree&p%5B%5D=facets.serviceability%5B%5D%3Dtrue&p%5B%5D=facets.brand%255B%255D%3DSamsung&otracker=nmenu_sub_TVs%20%26%20Appliances_0_Samsung")

# 페이지 로딩 대기
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'CGtC98')))

# URL을 저장할 리스트 초기화
url_list = []

# 요소 찾기
elements = driver.find_elements(By.CLASS_NAME, 'CGtC98')

# 각 요소에서 'href' 속성을 가져와 리스트에 추가 (sponsored 요소는 제외)
for element in elements:
    try:
        # 'f8qK5m' 클래스를 가진 자식 요소가 있는지 확인
        if element.find_elements(By.CLASS_NAME, 'f8qK5m'):
            # 'f8qK5m' 클래스를 가진 자식 요소가 있다면 skip
            continue
        # 자식 요소가 'f8qK5m' 클래스를 가지고 있지 않다면 href 추출
        url_value = element.get_attribute('href')
        if url_value:  # href 속성이 있는 경우만 추가
            url_list.append(url_value)
    except NoSuchElementException:
        # 자식 요소가 없으면 그냥 넘어가기
        continue 
        
# 리뷰 데이터를 저장할 리스트 초기화
all_reviews = []

for url in tqdm(url_list):
    try:
        driver.get(url)
        
        # XPath를 사용하여 제목 찾기. 제품 코드 별 XPATH 달라서 접근 불가... 제목에서 코드 추출하세용...
        try:
            # XPath를 사용하여 제목 찾기. 제품 코드 별 XPATH 달라서 접근 불가... 제목에서 코드 추출하세용... 
            list_items = driver.find_elements(By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[2]/div/div[1]/h1/span')
            
            # 텍스트 추출
            hp_data = [item.text for item in list_items]
            if not hp_data:
                hp_data = ["No data"]  # 데이터가 없을 경우 기본값
        except (NoSuchElementException, TimeoutException) as e:
            hp_data = ["Error fetching data"]  # 오류 발생 시 기본값
            
        # 리뷰 버튼 클릭 대기
        try:
            review_button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "_23J90q.RcXBOT")))
            if review_button.is_displayed():
                review_button.click()
        except (NoSuchElementException, TimeoutException):
            pass
        
        # 낮은 별점 접근
        try:
            low_rating_option = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#container > div > div.VLIitu > div > div > div.DOjaWF.gdgoEp.col-9-12 > div.DOjaWF.gdgoEp.col-12-12 > div > div:nth-child(2) > div > div > div > div > select > option:nth-child(4)")))
            low_rating_option.click()
        except (NoSuchElementException, TimeoutException) as e:
            print(f"Error selecting sort option for URL {url}: {e}")
        
        # 데이터 확보
        page_number = 1
        
        while True:
            try:
                # 페이지 URL 업데이트
                current_url = driver.current_url
                full_url = f'{current_url}&page={page_number}'
                driver.get(full_url)
                
                # ZmyHeo 클래스의 존재 여부를 확인
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ZmyHeo')))
                
                # 데이터 추출
                text_elements = driver.find_elements(By.CLASS_NAME, 'ZmyHeo')
                state_elements = driver.find_elements(By.CLASS_NAME, 'MztJPv')
                date_elements = driver.find_elements(By.CLASS_NAME, '_2NsDsF')
                star_elements = driver.find_elements(By.XPATH, "//*[contains(@class, 'XQDdHH') and (contains(@class, 'Js30Fc') or contains(@class, 'Czs3gR') or contains(@class, 'Ga3i8K'))]")
                
                text_list = [elem.text for elem in text_elements]
                state_list = [elem.text for elem in state_elements]
                date_list = [elem.text for elem in date_elements]
                dates = date_list[1::2]
                star_list = [elem.text for elem in star_elements]
                
                # 리뷰 데이터를 모든 리뷰 리스트에 추가
                for t, s, d, r in zip(text_list, state_list, dates, star_list):
                    for data in hp_data:
                        all_reviews.append({
                            'text': t,
                            'state': s,
                            'date': d,
                            'star': r,
                            'hp_data': data  # 각각의 hp_data 항목을 추가
                        })

                # 페이지 번호 증가
                page_number += 1
                
            except (NoSuchElementException, TimeoutException) as e:
                print(f"Error on page {page_number} for URL {full_url}: {e}")
                break
    
    except Exception as e:
        print(f"Error processing URL {url}: {e}")

# 현재 창 닫기
driver.quit()

  7%|█████▉                                                                             | 1/14 [00:46<09:58, 46.03s/it]

Error processing URL https://www.flipkart.com/samsung-80-cm-32-inch-hd-ready-led-smart-tizen-tv-bezel-free-design/p/itm33b1495b9e937?pid=TVSGHY2UZA9YHWQN&lid=LSTTVSGHY2UZA9YHWQNCPAW6M&marketplace=FLIPKART&store=ckf%2Fczl&spotlightTagId=BestsellerId_ckf%2Fczl&srno=b_1_3&otracker=browse&iid=ecc6a000-ccaf-42f0-9341-519e5dc0e992.TVSGHY2UZA9YHWQN.SEARCH&ssid=bbkdqs9v7k0000001724716326184: name 'NoSuchElementException' is not defined


 14%|███████████▊                                                                       | 2/14 [01:14<07:11, 35.98s/it]

Error processing URL https://www.flipkart.com/samsung-108-cm-43-inch-full-hd-led-smart-tizen-tv/p/itm1f514be8d5bad?pid=TVSGRS7GYVN3UZNZ&lid=LSTTVSGRS7GYVN3UZNZULGKSS&marketplace=FLIPKART&store=ckf%2Fczl&srno=b_1_4&otracker=browse&iid=ecc6a000-ccaf-42f0-9341-519e5dc0e992.TVSGRS7GYVN3UZNZ.SEARCH&ssid=bbkdqs9v7k0000001724716326184: name 'NoSuchElementException' is not defined


 21%|█████████████████▊                                                                 | 3/14 [02:16<08:42, 47.50s/it]

...

Error processing URL https://www.flipkart.com/samsung-80-cm-32-inch-hd-ready-led-tv/p/itm29e849f834662?pid=TVSFZRNNEEEXPGGG&lid=LSTTVSFZRNNEEEXPGGGIX210G&marketplace=FLIPKART&store=ckf%2Fczl&srno=b_1_20&otracker=browse&iid=ecc6a000-ccaf-42f0-9341-519e5dc0e992.TVSFZRNNEEEXPGGG.SEARCH&ssid=bbkdqs9v7k0000001724716326184: name 'NoSuchElementException' is not defined


 93%|████████████████████████████████████████████████████████████████████████████▏     | 13/14 [08:50<00:37, 37.07s/it]

Error processing URL https://www.flipkart.com/samsung-new-d-series-brighter-crystal-4k-vision-pro-2024-edition-125-cm-50-inch-ultra-hd-4k-led-smart-tizen-tv/p/itm84ebe7efb3a04?pid=TVSGYWKBERHVSKKB&lid=LSTTVSGYWKBERHVSKKBO0HVNH&marketplace=FLIPKART&store=ckf%2Fczl&srno=b_1_23&otracker=browse&iid=ecc6a000-ccaf-42f0-9341-519e5dc0e992.TVSGYWKBERHVSKKB.SEARCH&ssid=bbkdqs9v7k0000001724716326184: name 'NoSuchElementException' is not defined


100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [09:51<00:00, 42.22s/it]

Error processing URL https://www.flipkart.com/samsung-crystal-vision-4k-ismart-voice-assistant-163-cm-65-inch-ultra-hd-4k-led-smart-tizen-tv-iot-sensors-light-camera/p/itmc22cd15e47d2f?pid=TVSGRNGZ77Y3X2UH&lid=LSTTVSGRNGZ77Y3X2UHMOXPN3&marketplace=FLIPKART&store=ckf%2Fczl&srno=b_1_24&otracker=browse&iid=ecc6a000-ccaf-42f0-9341-519e5dc0e992.TVSGRNGZ77Y3X2UH.SEARCH&ssid=bbkdqs9v7k0000001724716326184: name 'NoSuchElementException' is not defined


In [4]:
df_SS_TV = pd.DataFrame(all_reviews)
df_SS_TV

,text,state,date,star,hp_data
0,PLEASE DO NOTE THAT: THE TV DOES NOT COME WITH...,"Certified Buyer, New Town","Sep, 2021",1,SAMSUNG 80 cm (32 Inch) HD Ready LED Smart Tiz...
1,"The picture quality is very very bad, I didn't...","Certified Buyer, Dindigul",10 days ago,1,SAMSUNG 80 cm (32 Inch) HD Ready LED Smart Tiz...
2,Very bad experience automatic switch off/on al...,"Certified Buyer, Noida",18 days ago,1,SAMSUNG 80 cm (32 Inch) HD Ready LED Smart Tiz...
3,I ordered but in 20 days screen went blurring....,"Certified Buyer, Hosapete",19 days ago,1,SAMSUNG 80 cm (32 Inch) HD Ready LED Smart Tiz...
4,Nice products,"Certified Buyer, Sheohar District",28 days ago,1,SAMSUNG 80 cm (32 Inch) HD Ready LED Smart Tiz...
...,...,...,...,...,...
2289,Good,"Certified Buyer, Mahabubnagar",10 months ago,4,SAMSUNG Crystal Vision 4K iSmart with Voice As...
2290,I am writing after one week using this tv and ...,"Certified Buyer, Bengaluru",10 months ago,4,SAMSUNG Crystal Vision 4K iSmart with Voice As...
2291,This product is very good. But in this TV inbu...,"Certified Buyer, Bhubaneswar",10 months ago,4,SAMSUNG Crystal Vision 4K iSmart with Voice As...
2292,nice product and picture and sound quality ver...,"Certified Buyer, Katihar",10 months ago,4,SAMSUNG Crystal Vision 4K iSmart with Voice As...


In [5]:
df_SS_TV.to_csv("df_SS_TV.csv")